In [221]:
import pandas as pd
import openpyxl
import numpy as np
import math

In [222]:

# Sample DataFrame

df = pd.read_csv('/home/igor/Desafio-VLGI/Desafio-VLGI/data/fake_position.csv')



In [223]:

al = pd.read_excel('/home/igor/Desafio-VLGI/Desafio-VLGI/data/fake_allocation_policies.xlsx')

In [224]:
al

,classe,conservador,moderado-conservador,moderado,moderado-agressivo,agressivo
0,Renda Fixa Pós-Fixada,0.70,0.46,0.30,0.15,0.05
1,Renda Fixa Inflação,0.12,0.16,0.24,0.20,0.19
2,Renda Fixa Pré-Fixada,0.05,0.08,0.10,0.11,0.10
3,Renda Variável,0.02,0.04,0.07,0.14,0.20
4,Multimercado,0.09,0.21,0.22,0.30,0.31
5,Alternativos,0.00,0.00,0.00,0.00,0.00
6,Internacional,0.02,0.05,0.07,0.10,0.15
7,Saldo em Conta,0.00,0.00,0.00,0.00,0.00


In [225]:
# Realiza uma filtragem somando cada classe de investimento por conta e reordena os index e renomeia a coluna position_value para Soma. E filtra para aparecer somente as colunas account_code, account_suitability, class_name e soma.
#df['account_suitability'].fillna(value = 'indefinido', inplace = True)
df['account_suitability'].fillna(value = 'moderado', inplace = True)
df['asset_cnpj'].fillna(value = 0, inplace = True)

df = df.groupby(['account_code', 'account_suitability','class_name'])['position_value'].sum().reset_index().sort_values(['account_code', 'class_name']).rename(columns={'position_value' : 'Soma'})


In [226]:
df

,account_code,account_suitability,class_name,Soma
0,2429,moderado,Renda Fixa Inflação,1.343101e+08
1,2429,moderado,Renda Fixa Pré-Fixada,2.968725e+05
2,2429,moderado,Renda Fixa Pós-Fixada,6.425970e+03
3,2429,moderado,Reserva de Liquidez,1.306173e+04
4,2429,moderado,Saldo em Conta,0.000000e+00
...,...,...,...,...
149,82356,conservador,Renda Fixa Inflação,2.742888e+04
150,82356,conservador,Renda Fixa Pré-Fixada,3.144487e+04
151,82356,conservador,Renda Fixa Pós-Fixada,3.742832e+04
152,82356,conservador,Reserva de Liquidez,2.904750e+05


In [227]:
#Lista com os clientes
clients = df['account_code'].unique()
clients

array([ 2429,  6777,  7826, 18073, 20921, 32263, 32449, 45848, 46168,
       46744, 47497, 49232, 51239, 55101, 55836, 56165, 58230, 65547,
       66740, 72740, 73707, 76497, 78059, 82356])

In [228]:
#  Realiza a soma dos valores investidos de um cliente e armazena em uma variável: total
total = (df[df['account_code'] == clients[0]])['Soma'].sum()
print(total)    

134626482.6309


In [229]:
client0 = df[df['account_code'] == clients[0]]
df['Soma'] = client0['Soma'].div(total)

In [230]:
# Filtrando respectivo cliente em um dataframe separado
teste = df[df['account_code'] == clients[0]]
teste


,account_code,account_suitability,class_name,Soma
0,2429,moderado,Renda Fixa Inflação,0.997650
1,2429,moderado,Renda Fixa Pré-Fixada,0.002205
2,2429,moderado,Renda Fixa Pós-Fixada,0.000048
3,2429,moderado,Reserva de Liquidez,0.000097
4,2429,moderado,Saldo em Conta,0.000000


In [231]:
perfil = teste['account_suitability'].iloc[0] # perfil do cliente a partir da tabela de alocações
class_allocation = pd.Series(al.classe.values, index=al[perfil].values).to_dict() # dicionário com as alocações do respectivo cliente
class_allocation = {v: k for k, v in class_allocation.items()} # inverte as chaves e os valores do dicionário
print(class_allocation)

{'Renda Fixa Pós-Fixada': 0.3, 'Renda Fixa Inflação': 0.24, 'Renda Fixa Pré-Fixada': 0.1, 'Internacional': 0.07, 'Multimercado': 0.22, 'Saldo em Conta': 0.0}


In [232]:
#Dicionario do percentual de tipo de investimento do cliente respectivo
client = dict(zip(teste['class_name'], teste['Soma']))
print(client)

{'Renda Fixa Inflação': 0.9976500890945258, 'Renda Fixa Pré-Fixada': 0.00220515706789966, 'Renda Fixa Pós-Fixada': 4.773184201519862e-05, 'Reserva de Liquidez': 9.702199555945332e-05, 'Saldo em Conta': 0.0}


In [233]:
#Calculo da aderência do cliente ao perfil de investimento
distance = 0
for asset_class, allocation_percentage in class_allocation.items():
    client_allocation = client.get(asset_class, 0)
    distance += (allocation_percentage - client_allocation) ** 2

euclidean_distance = math.sqrt(distance)

aderencia = dict()

aderencia.update({f"{teste['account_code'].iloc[0]}" : f"{euclidean_distance}"})
print(aderencia)

{'2429': '0.8525660396603754'}
